In [1]:
import pyscal.crystal_structures as pcs
import pyscal.csystem as pcm
import pyscal.core as pc
from ase import atoms
from ase.io import read, write
from ase.build import bulk
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nx = 10
satoms, box = pcs.make_crystal(structure='bcc', 
    lattice_constant=3.127, 
    repetitions=(nx,nx,nx),)
print(len(satoms["positions"]))
sys = pc.System()
sys.box = box
sys.atoms = satoms


2000


In [3]:
sys.find_neighbors(method="cutoff", 
                   cutoff=3.6)

In [10]:
theta_arr = sys.atoms["theta"][0]
phi_arr = sys.atoms["phi"][0]

In [115]:
xx = pcm.calculate_q_atom(4, [0.9553166181245093], [0.7853981633974483])

In [12]:
from scipy.special import sph_harm

In [229]:
def calc_sph(atoms, l=2):
    theta = np.array(sys.atoms["theta"])
    phi = np.array(sys.atoms["phi"])
    shs = []
    sh = sph_harm(0, l, phi, theta)
    shs.append(np.mean(sh, axis=1))
    for m in range(1, l+1):
        sh = sph_harm(m, l, phi, theta)
        shs.append(np.mean(sh, axis=1))
        sh = sh*(-1)**(-m)
        shs.append(np.mean(sh, axis=1))
    shs = np.array(shs)
    q_real = np.real(shs)
    q_imag = np.imag(shs)
    shs_sum = np.sum(q_real**2, axis=0) + np.sum(q_imag**2, axis=0)
    factor = (4.0*np.pi/(2*l+1))
    qval = (factor*shs_sum)**0.5
    
    return qval, q_real, q_imag

In [230]:
%%timeit
shs = calc_sph(sys.atoms, l=12)

79.5 ms ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
